In [ ]:
!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 3.4 MB 36.1 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=7d444b78edb3f6a216bc2f886b491e104a4c110b419de8324b8073c0510c32f9
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [ ]:
import pysat.solvers
from pysat.card import CardEnc as encode
from random import shuffle
import numpy as np
from math import sqrt
from tqdm import tqdm 
import matplotlib.pyplot as plt

Rule = [[3,], [2,3]]
nxn = 21
Population = 233 
Enc = 0

rulestr = lambda r: 'B' + ''.join(map(str, r[0])) + '/S' + ''.join(map(str, r[1])) + '\n'

def draw(arr):
    # cmaps =  ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
    #                     'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
    #                     'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper']
    cmap = 'copper'
    size = 3
    plt.figure(num = None, figsize=(size, size), dpi=300)
    plt.axis('off')
    plot = plt.imshow(arr, cmap = cmap)
    plt.show()
    plt.close()

def rle_encode(data):
    encoding = ''
    prev_char = ''
    count = 1
    if not data: return ''
    for char in data:
        if char != prev_char:
            if prev_char:
                encoding = encoding + str(count) + prev_char if count > 1 else encoding + prev_char
            count = 1
            prev_char = char
        else:
            count += 1
    else:
        encoding = encoding + str(count) + prev_char if count > 1 else encoding + prev_char
        return encoding

def clauses_or(A, B, top):
    x = top + 1 
    y = top + 2

    out = [[x,y],]

    for i in A:
        out.append(i + [-x,])

    for i in B:
        out.append(i + [-y,])

    return out

def clauses_not(A, top):
    x = [top + i for i in range(1, len(A)+1)] 
    out = [[-i for i in x],]

    for k, v in enumerate(A):
        out.append(v + [-x[k],])

    return out, max(x) + 1

def m_to_rle(m):
    
    prefix = 'x = 0, y = 0, rule = ' + rulestr(Rule)
    m = [''.join(list(map(str, i)) + ['$',]) for i in m]
    S = ''.join(m)
    S = S.replace('0', 'b')
    S = S.replace('1', 'o')
    return rle_encode(prefix + S + '!')


nn = nxn + 2
dim = (nn, nn)
N = dim[0]*dim[1]
Rr = Population
v = []
t = 0
d = dict()

for i in range(dim[0]):
    for j in range(dim[1]):
        t += 1
        d[(i,j)] = t

        v.append((i,j))

clauses = []
topid = dim[0]*dim[1]+1

for a, b in v:
    g  = d[(a,b)]

    temp = [(a+1,b),(a,b+1),  (a-1,b),(a,b-1),   (a-1,b-1),(a+1,b+1), (a-1,b+1),(a+1,b-1)]
    #temp = [ (a+1,b),(a-1,b), (a,b+1),(a,b-1), ]

    if a == dim[0]-1:
        temp = list(filter(lambda x: x not in ((a+1,b),(a+1,b+1),(a+1,b-1)), temp))

    if b == dim[1]-1:
        temp = list(filter(lambda x: x not in ((a,b+1),(a-1,b+1),(a+1,b+1)), temp))

    if a == 0:
        temp = list(filter(lambda x: x not in ((a-1,b),(a-1,b-1),(a-1,b+1)), temp))

    if b == 0:
        temp = list(filter(lambda x: x not in ((a,b-1),(a+1,b-1),(a-1,b-1)), temp))

        
    tempc = []
    c = [d[k] for k in temp]

    if len(c) > 5:
        tclaus = encode.atleast(lits = c, bound = Rule[1][0], top_id = topid).clauses
        for i in tclaus:
            clauses.append([-g,] +  i)
        topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)


        tclaus = encode.atmost(lits = c, bound = Rule[1][-1], top_id = topid).clauses
        for i in tclaus:
            clauses.append([-g,] +  i)
        topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)

    temp1 = []
    temp2 = []

    if len(c) > 3:

        for Item_rule in Rule[0]:
            if len(c) >= Item_rule+1:

                temp1 = []
                temp2 = []

                if Item_rule < 8:
                    tclaus = encode.atleast(lits = c, bound = Item_rule+1, top_id = topid).clauses
                    for i in tclaus:
                        temp1.append([ g, ] +  i)
                    topid = max(max(abs(literal) for clause in clauses + temp1 for literal in clause), topid)

                    tclaus = encode.atmost(lits = c, bound = Item_rule-1, top_id = topid).clauses
                    for i in tclaus:
                        temp2.append([ g, ] +  i)
                    topid = max(max(abs(literal) for clause in clauses + temp2 + temp1 for literal in clause), topid)

                    for i in clauses_or(temp1, temp2, topid):
                        clauses.append(i)
                    topid += 2
                # else:
                #     tclaus = encode.atmost(lits = c, bound = Item_rule-1, top_id = topid).clauses
                #     for i in tclaus:
                #         clauses.append([ g, ] +  i)
                #     topid = max(max(abs(literal) for clause in clauses  for literal in clause), topid)

    # topid += 3

    if len(c) < 8:
        clauses.append([-g, ] )
        topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)

prArr = []
clausesTemp = clauses + encode.equals(lits=[i for i in range(1,dim[0]*dim[1]+1)], top_id = topid, bound = Rr).clauses
while 1:
    solver = pysat.solvers.Cadical()
    for clause in clausesTemp:
        solver.add_clause(clause)
    sat = solver.solve()
    

    if sat:
        model = solver.get_model()
        temp = [1 if i> 0 else 0 for i in model[:N]]
        clausesTemp.append([-i for i in model[:N]])
    
        print( 'nxn: ', nn-2, 'population: ', sum(temp))
        temp = np.array(temp)
        temp.resize(dim)
        print(m_to_rle(temp.tolist()))
        draw(temp) 
    else:
        print('-------------------------DONE-------------------------')
        break
    